In [2]:
#Import URLs and UIDs into pandas table - allow to append additional data later
# if re.search(regex, line) -> allows for searches based on regular expressions
#sys.exit(1) - will exit the running program
#
#from pprint import pprint -> pretty print
#
#import pickle ; pickle.dump(list,out_file)  *requires import sys
#saves data as a pickle file, which is an object itself, and you can also use pickle.load(in_file) -> and it will allow you
#to open/load data in the order in which it was originally saved/pickled.  FIFO
#
#JSON - 
#import json
#store dictionaries as JSON files ->with open('data.json', 'w') as f: \r json.dump(data.f)  **use indent function to make the
#data more easily readable by human beings
#
#Look at the class code for reading/writing CSV files - It will be WAY easier!!! import csv
#when crawling: import time time.sleep(10) - will pause for 10 seconds

#Download PDF Files to disk

import PyPDF2
import sys
import collections
import urllib2
from StringIO import StringIO
from django.utils.encoding import smart_str, smart_unicode
import datetime
import pandas as pd
import csv as csv
from __future__ import print_function

start_time = datetime.datetime.now().time()
print("Start: "+str(start_time))

def convertPyPDFDate(origDate):
    try:
        return (origDate[2:6]+"-"+origDate[6:8]+"-"+origDate[8:10])
    except:
        return origDate

def recreateTsvFile():
    urldf = pd.read_csv("PDFs.csv")  #Import List of PDF Files from CSV
    urldf["URL"][:10] #first 10 rows of column "URL"
    urldf["URL"].value_counts() #visual check for duplicates
    urldf["UID"] = ""  
    urldf["Download"] = "" 
    urldf["CreatedDate"] = ""
    urldf["ModDate"] = ""
    urldf["Creator"] = ""
    urldf["Producer"] = ""
    urldf["Author"] = ""
    urldf["Title"] = ""

    for x in range (0, len(urldf)):
        urldf["UID"][x] = "BSA_"+str(x+101)  #create UID for each PDF FIle

    urldf.to_csv("D:\BSA_PDF_Files\URLs_Updated.txt", sep="\t") #write to new csv file the URL & UID for each
    

def downloadPdfAsBin(urldf):
    folder = "D:\BSA_PDF_Files\Bin\\"
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

    for x in range(0, len(urldf)):
        tempReq = urllib2.Request(urldf["URL"][x], headers=hdr)
        try:
            tempPage = urllib2.urlopen(tempReq)    
        except urllib2.HTTPError, e:
            print(e.fp.read())
    
        tempFile = open(folder+urldf["UID"][x]+".pdf", 'wb')
        tempFile.write(tempPage.read())
        tempFile.close()
#downloadPdfAsBin(urldf[:10]) 

def downloadPdfAsIs(urldf):

    folder = "D:\BSA_PDF_Files\\"
    
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

    for x in range(0, len(urldf)):
        try:
            if pd.isnull(urldf["Download"][x]):
                tempURL = str(urldf["URL"][x])
                tempReq = urllib2.Request(tempURL, headers=hdr)
                try:
                    tempPage = urllib2.urlopen(tempReq)
                    urldf["Download"][x] = "Success"
                except urllib2.HTTPError, e:
                    urldf["Download"][x] = e.fp.read()
                    continue
                tempPdf = PyPDF2.PdfFileReader(StringIO(tempPage.read()))
            
                if tempPdf.isEncrypted: 
                    try: tempPdf.decrypt('')
                    except: print("Attempted decrpytion failed on line: "+str(x))
            
                merger = PyPDF2.PdfFileMerger()
                try: merger.append(tempPdf)
                except:
                    urldf["Download"][x]="Outfile Error: No Content in File"+str(x)
                    continue
            
                try: docInfoLen = len(tempPdf.documentInfo)
                except:
                    urldf["Download"][x]="Encrpytion Error: Couldnt Decrypt Metadata on File"+str(x)
                    continue
            
                if docInfoLen>0:
                    try:merger.addMetadata(tempPdf.documentInfo)
                    except: urldf["Download"][x]="Metadata error: Metadata not written to file"+str(x)
                    
                    try: urldf["CreatedDate"][x] = convertPyPDFDate(tempPdf.documentInfo["/CreationDate"])
                    except: urldf["CreatedDate"][x] = ""
                
                    try: urldf["ModDate"][x] = convertPyPDFDate(tempPdf.documentInfo["/ModDate"])
                    except: urldf["ModDate"][x] = ""
            
                    try: urldf["Creator"][x] = tempPdf.documentInfo["/Creator"]
                    except: urldf["Creator"][x] = ""
            
                    try: urldf["Producer"][x] = tempPdf.documentInfo["/Producer"]
                    except:urldf["Producer"][x] = ""
            
                    try: urldf["Author"][x] = tempPdf.documentInfo["/Author"]
                    except:urldf["Author"][x] = ""
                
                    try: urldf["Title"][x] = tempPdf.documentInfo["/Title"]
                    except:urldf["Title"][x] = ""
        
        
                outFile= open(folder+urldf["UID"][x]+".pdf", 'wb')
                try: merger.write(outFile)
                except: 
                    outFile.write("Error writing file. Please Manually Download")
                    urldf["Download"][x]="Outfile Error: No Content in File"+str(x)
                outFile.close()
            else:
                #print(str(x), end=", ")
                continue
        except:
            urldf.to_csv("D:\BSA_PDF_Files\URLs_Updated.txt",sep="\t",header=True, index=False, encoding='utf-8')
    urldf.to_csv("D:\BSA_PDF_Files\URLs_Updated.txt",sep="\t",header=True, index=False, encoding='utf-8')

###################    
### Run Process ###
###################
#recreateTsvFile()
urldf = pd.read_csv("D:\BSA_PDF_Files\URLs_Updated.txt", sep="\t")
downloadPdfAsIs(urldf)
#print(urldf)       
end_time = datetime.datetime.now().time()
print("End: "+str(end_time))



############################################### 
### Work Still to be done to the above code ###
###############################################
#







Start: 10:08:43.183000


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy [__main__.py:99]
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy [__main__.py:124]
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy [__main__.py:127]
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy [__main__.py:130]
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-v

End: 10:23:43.416000
